In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("titanic.csv")

In [9]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


# Calculate Survival Rate by Passenger Class:# Example 1: Single aggregation

In [10]:
df.groupby("Pclass")['Survived'].mean()

Pclass
1    0.467290
2    0.322581
3    0.330275
Name: Survived, dtype: float64

# Calculate average age by passenger class and gender: # Example 2: Multi aggregation

In [11]:
df.groupby(['Pclass','Sex'])['Age'].mean()

Pclass  Sex   
1       female    41.333333
        male      40.520000
2       female    24.376552
        male      30.940678
3       female    23.073400
        male      24.525104
Name: Age, dtype: float64

# Count Passengers by Embarked Port:

In [12]:
df.groupby("Embarked")['PassengerId'].count()

Embarked
C    102
Q     46
S    270
Name: PassengerId, dtype: int64

# Find the oldest and youngest passengers in each passenger class

In [13]:
df.groupby("Pclass")['Age'].agg(['max','min'])

,max,min
Pclass,,
1,76.0,6.00
2,63.0,0.92
3,60.5,0.17


# Calculate total fare and average age for each embarked port

In [14]:
df.groupby("Embarked").agg({'Fare':'sum','Age':'mean'})

,Fare,Age
Embarked,,
C,6758.4960,34.737805
Q,504.0542,29.318182
S,7593.9874,28.758772


# Find the Most Common Embarked Port for each Passenger Class:

In [16]:
df.groupby("Pclass")['Embarked'].apply(lambda x: x.mode().iloc[0])

Pclass
1    C
2    S
3    S
Name: Embarked, dtype: object

# Calculate the Age Range (Max - Min) for each Title (Mr, Mrs, etc.):

In [13]:
df['title'] = df['Name'].str.extract('([A-Za-z]+)\.')

In [14]:
df.groupby("title")['Age'].agg(lambda x:x.max() - x.min())

title
Col        6.00
Dona       0.00
Dr         0.00
Master    14.17
Miss      44.83
Mr        53.00
Mrs       60.00
Ms          NaN
Rev       11.00
Name: Age, dtype: float64

# Count the Number of Passengers Survived and Died for each Passenger Class and Embarked Port:

In [15]:
df.groupby(['Pclass','Embarked','Survived'])['PassengerId'].count()

Pclass  Embarked  Survived
1       C         0            28
                  1            28
        Q         1             1
        S         0            29
                  1            21
2       C         0             7
                  1             4
        Q         0             4
        S         0            52
                  1            26
3       C         0            27
                  1             8
        Q         0            18
                  1            23
        S         0           101
                  1            41
Name: PassengerId, dtype: int64

# Calculate the Fare Range (Max - Min) for each Passenger Class and Gender:

In [16]:
df.groupby(['Pclass','Sex'])['Fare'].agg(lambda x: x.max() - x.min() )

Pclass  Sex   
1       female    486.6292
        male      262.3750
2       female     54.5000
        male       63.8125
3       female     62.6000
        male       66.3792
Name: Fare, dtype: float64

# Group Passengers by Age Range and Calculate the Survival Rate:

In [17]:
# Group passengers by age range and calculate survival rate
age_bins = [0, 18, 30, 50, 100]
age_labels = ['0-18', '19-30', '31-50', '51+']
df['AgeRange'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels)

In [19]:
df.groupby('AgeRange')['Survived'].mean() * 100

AgeRange
0-18     44.444444
19-30    36.301370
31-50    34.653465
51+      48.387097
Name: Survived, dtype: float64

# Calculate the Cumulative Survival Rate over Time:

In [19]:
df_sorted = df.sort_values("Age")

In [20]:
df_sorted['c_age'] = df_sorted['Survived'].cumsum()

In [21]:
df_sorted.groupby('c_age')['Survived'].max()

c_age
1      1
2      1
3      1
4      1
5      1
      ..
148    1
149    1
150    1
151    1
152    1
Name: Survived, Length: 152, dtype: int64

# Find the Passengers with the Highest Fare in each Age Group:

In [22]:
# Create age bins and labels
age_bins = [0, 18, 30, 50, 100]
age_labels = ['0-18', '19-30', '31-50', '51+'] 
df['agegroup'] = pd.cut(df['Age'],bins=age_bins,labels=age_labels)

In [25]:
df.loc[df.groupby('agegroup')['Fare'].idxmax()][['PassengerId','Fare']]

,PassengerId,Fare
64,956,262.3750
53,945,263.0000
24,916,262.3750
343,1235,512.3292


# Calculate the Survival Rate for Different Family Sizes:

In [30]:
# Create a new column 'FamilySize' by summing 'SibSp' and 'Parch'
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# Group the data by 'FamilySize' and calculate the survival rat
df.groupby('FamilySize')['Survived'].mean() * 100

FamilySize
1     26.877470
2     48.648649
3     52.631579
4     71.428571
5     28.571429
6     66.666667
7     25.000000
8     50.000000
11    50.000000
Name: Survived, dtype: float64

# Find the Most Common Ticket Number for each Passenger Name Prefix:

In [31]:
# Extract passenger name prefix from 'Name'
df['NamePrefix'] = df['Name'].str.extract(' ([A-Za-z]+)\.')

# Find the most common ticket number for each passenger name prefix
df.groupby('NamePrefix')['Ticket'].apply(lambda x: x.mode().iloc[0])

NamePrefix
Col             113780
Dona          PC 17758
Dr               33638
Master          347077
Miss          PC 17608
Mr          C.A. 31029
Mrs       F.C.C. 13534
Ms              364856
Rev             237393
Name: Ticket, dtype: object

# Calculate the Median Fare for Passengers Travelling Alone vs. with Family:

In [70]:
# Create a new column 'Alone' to indicate passengers travelling alone
df['Alone'] = (df['SibSp'] + df['Parch']) == 0

# Group the data by 'Alone' and calculate the median fare
median_fare_by_alone = df.groupby('Alone')['Fare'].median()

In [71]:
median_fare_by_alone

Alone
False    26.00
True      8.05
Name: Fare, dtype: float64

# Find the Passenger with the Highest Fare in each Passenger Class:



In [73]:
# Find the passenger with the highest fare in each passenger class
df.loc[df.groupby('Pclass')['Fare'].idxmax()][['Pclass','Fare']]

,Pclass,Fare
343,1,512.3292
212,2,73.5000
188,3,69.5500


# Calculate the Average Fare for each Combination of Passenger Class and Embarked Port:

In [74]:
# Group the data by 'Pclass' and 'Embarked' and calculate the average fare
average_fare_by_class_embarked = df.groupby(['Pclass', 'Embarked'])['Fare'].mean()

In [75]:
average_fare_by_class_embarked

Pclass  Embarked
1       C           110.073511
        Q            90.000000
        S            76.677504
2       C            20.120445
        Q            11.273950
        S            23.056090
3       C            10.658700
        Q             8.998985
        S            13.913030
Name: Fare, dtype: float64